In [1]:
pip install tabulate


In [ ]:
import random
import matplotlib.pyplot as plt
import numpy as np
import math
from tabulate import tabulate
from matplotlib.patches import Patch
from collections import defaultdict

# Initial settings
random.seed(42)
SPEED_OF_PROPAGATION = 2e8  # Speed of light in fiber optic (m/s)
CPU_THRESHOLD = 100000

class Node:
    def __init__(self, id, is_cloud=True):
        self.id = id
        self.is_cloud = is_cloud
        self.current_time = 0.0
        self.busy_time = 0.0
        self.task_queue = []

        # Initial resources
        if is_cloud:
            self.cpu_rate = random.uniform(30000, 50000)
            self.total_ram = random.uniform(16000, 32000)
            self.total_storage = random.uniform(80000, 200000)
        else:
            self.cpu_rate = random.uniform(10000, 20000)
            self.total_ram = random.uniform(8000, 16000)
            self.total_storage = random.uniform(20000, 80000)

        self.available_cpu = self.cpu_rate
        self.available_ram = self.total_ram
        self.available_storage = self.total_storage

        # Network specifications
        self.bandwidth = random.uniform(500, 1000) if is_cloud else random.uniform(100, 500)

        # Costs and energy
        self.cpu_cost = random.uniform(0.2, 0.5) if is_cloud else random.uniform(0.1, 0.3)
        self.memory_cost = random.uniform(0.000005, 0.000015)
        self.bandwidth_cost = random.uniform(0.01, 0.03)
        self.energy_cost = random.uniform(0.05, 0.1)
        self.power_consumption = random.uniform(100, 500) if is_cloud else random.uniform(50, 200)
        self.idle_power = self.power_consumption * 0.3

        # ** Location setup:
        # Cloud nodes are placed far from users (y=5000-5050)
        # Fog nodes are placed near users (y=200-250)
        # This affects propagation delay in calculate_transfer_time()
        # NEW: Location impacts network latency between nodes and tasks
        if is_cloud:
            self.location = (random.uniform(0, 50), random.uniform(5000, 5050))
        else:
            self.location = (random.uniform(0, 50), random.uniform(200, 250))

        # Statistics
        self.completed_tasks = []
        self.waiting_times = []
        self.processing_times = []
        self.total_cost = 0.0
        self.total_energy = 0.0

    def can_accept_task(self, task):
        """Check if task can be accepted with load management"""
        required_cpu_ratio = task.cpu / self.cpu_rate
        required_ram_ratio = task.ram / self.total_ram
        required_storage_ratio = task.storage / self.total_storage

        # Prevent unrealistic allocation
        max_single_allocation = 0.7  # Max 70% resources to single task
        if (required_cpu_ratio > max_single_allocation or
            required_ram_ratio > max_single_allocation or
            required_storage_ratio > max_single_allocation):
            return False

        return (self.available_cpu >= task.cpu and
                self.available_ram >= task.ram and
                self.available_storage >= task.storage)

    def calculate_transfer_time(self, task):
        """Calculate data transfer time"""
        # ** Distance calculation between node and task location
        # Uses Euclidean distance which affects propagation delay
        # NEW: Network delay has two components:
        # 1. Propagation delay (distance/speed)
        # 2. Transmission delay (data size/bandwidth)
        distance = math.sqrt((self.location[0]-task.location[0])**2 +
                           (self.location[1]-task.location[1])**2)
        propagation_delay = distance / SPEED_OF_PROPAGATION
        transmission_delay = (task.data_size * 8) / (self.bandwidth * 0.125)  # MB to Mb and Mbps to MB/s
        return propagation_delay, transmission_delay

    def assign_task(self, task, arrival_time):
        """Assign task with resource management"""
        prop_delay, trans_delay = self.calculate_transfer_time(task)
        transfer_time = prop_delay + trans_delay
        arrival_at_node = arrival_time + transfer_time

        # FCFS scheduling considering previous tasks' finish time
        start_processing = max(self.current_time, arrival_at_node)
        processing_time = task.cpu / self.cpu_rate
        finish_time = start_processing + processing_time

        # Store task info
        task_record = {
            'task': task,
            'arrival_time': arrival_time,
            'transfer_time': transfer_time,
            'start_processing': start_processing,
            'finish_time': finish_time,
            'processing_time': processing_time,
            'waiting_time': start_processing - arrival_at_node
        }

        self.task_queue.append(task_record)
        self.current_time = finish_time
        self.busy_time += processing_time

        # Reserve resources
        self.available_cpu -= task.cpu
        self.available_ram -= task.ram
        self.available_storage -= task.storage

        return finish_time

    def complete_task(self, task_record):
        """Release resources after task completion"""
        task = task_record['task']

        # Release resources
        self.available_cpu += task.cpu
        self.available_ram += task.ram
        self.available_storage += task.storage

        # Calculate costs
        cpu_cost = task_record['processing_time'] * self.cpu_cost
        memory_cost = task.ram * task_record['processing_time'] * self.memory_cost
        bandwidth_cost = task.data_size * self.bandwidth_cost

        # Calculate energy
        energy = (self.power_consumption * task_record['processing_time']) / 3600 / 1000  # kWh

        # Store statistics
        self.completed_tasks.append(task_record)
        self.waiting_times.append(task_record['waiting_time'])
        self.processing_times.append(task_record['processing_time'])
        self.total_cost += cpu_cost + memory_cost + bandwidth_cost
        self.total_energy += energy

class Task:
    def __init__(self, id, arrival_time):
        self.id = id
        self.cpu = random.uniform(5000, 150000)  # More realistic range
        self.ram = random.uniform(50, 1000)
        self.storage = random.uniform(20, 2000)
        self.data_size = random.uniform(1, 200)
        self.location = (random.uniform(0, 200), random.uniform(0, 30))
        self.arrival_time = arrival_time

def simulate_cloud_fog():
    # Create nodes
    cloud_nodes = [Node(i, True) for i in range(10)]
    fog_nodes = [Node(i+10, False) for i in range(10)]
    nodes = cloud_nodes + fog_nodes

    # Create tasks with exponential arrival times
    num_tasks = 200
    inter_arrival_times = np.random.exponential(0.5, num_tasks)
    arrival_times = np.cumsum(inter_arrival_times)
    tasks = [Task(i, arrival_times[i]) for i in range(num_tasks)]

    # Task assignment
    for task in tasks:
        # Select suitable nodes based on CPU threshold
        if task.cpu > CPU_THRESHOLD:
            candidates = [n for n in nodes if n.is_cloud and n.can_accept_task(task)]
        else:
            candidates = [n for n in nodes if not n.is_cloud and n.can_accept_task(task)]

        # If no suitable node, check all nodes
        if not candidates:
            candidates = [n for n in nodes if n.can_accept_task(task)]

        # If still no node, wait for resources to free up
        while not candidates:
            # Advance time to free resources
            min_finish_time = min(n.current_time for n in nodes)
            for n in nodes:
                if n.current_time == min_finish_time and n.task_queue:
                    completed_task = n.task_queue.pop(0)
                    n.complete_task(completed_task)

            # Check again
            if task.cpu > CPU_THRESHOLD:
                candidates = [n for n in nodes if n.is_cloud and n.can_accept_task(task)]
            else:
                candidates = [n for n in nodes if not n.is_cloud and n.can_accept_task(task)]

        # Select node with earliest finish time (Load Balancing)
        best_node = min(candidates, key=lambda n: n.current_time)
        best_node.assign_task(task, task.arrival_time)

    # Complete remaining tasks
    for node in nodes:
        while node.task_queue:
            completed_task = node.task_queue.pop(0)
            node.complete_task(completed_task)

    # Calculate system metrics
    makespan = max(n.current_time for n in nodes)
    total_cost = sum(n.total_cost for n in nodes)
    total_energy = sum(n.total_energy for n in nodes)

    # Calculate idle time and add to energy
    for node in nodes:
        node.idle_time = makespan - node.busy_time
        idle_energy = (node.idle_power * node.idle_time) / 3600 / 1000
        node.total_energy += idle_energy
        node.total_cost += idle_energy * node.energy_cost

    # Calculate wait statistics
    all_waiting_times = [wt for n in nodes for wt in n.waiting_times]
    avg_waiting_time = np.mean(all_waiting_times) if all_waiting_times else 0
    max_waiting_time = max(all_waiting_times) if all_waiting_times else 0

    # Display results
    print("\n=== System Metrics ===")
    print(f"Makespan: {makespan:.2f} seconds")
    print(f"Total cost: ${total_cost:.2f}")
    print(f"Total energy: {total_energy:.4f} kWh")
    print(f"Average task waiting time: {avg_waiting_time:.2f} seconds")
    print(f"Maximum waiting time: {max_waiting_time:.2f} seconds")

    # Display node statistics
    print("\n=== Node Statistics ===")
    print("ID | Type   | Tasks | Finish Time | CPU Usage | Cost | Energy")
    print("-"*70)
    for node in nodes:
        cpu_util = (node.busy_time / makespan * 100) if makespan > 0 else 0
        print(f"{node.id:2} | {'Cloud' if node.is_cloud else 'Fog':5} | {len(node.completed_tasks):6} | "
              f"{node.current_time:10.2f} | {cpu_util:10.1f}% | "
              f"${node.total_cost:6.2f} | {node.total_energy:.4f}")

    # Plot graphs
    plot_system_metrics(nodes, tasks)
    plot_resource_utilization(nodes)
    plot_waiting_time_distribution(nodes)

def plot_system_metrics(nodes, tasks):
    plt.figure(figsize=(15, 10))

    # Gantt chart
    colors = plt.cm.tab20(np.linspace(0, 1, len(nodes)))
    for i, node in enumerate(nodes):
        for task in node.completed_tasks:
            # Transfer time
            plt.barh(i, task['transfer_time'],
                    left=task['arrival_time'],
                    color='lightblue', edgecolor='black')

            # Waiting time
            if task['waiting_time'] > 0:
                plt.barh(i, task['waiting_time'],
                        left=task['arrival_time'] + task['transfer_time'],
                        color='lightgray', edgecolor='black')

            # Processing time
            plt.barh(i, task['processing_time'],
                    left=task['start_processing'],
                    color=colors[i], edgecolor='black')

    plt.xlabel('Time (seconds)')
    plt.ylabel('Node ID')
    plt.title('Task Scheduling Gantt Chart')
    plt.yticks(range(len(nodes)),
              [f'Node {n.id} ({"Cloud" if n.is_cloud else "Fog"})' for n in nodes])

    legend_elements = [
        Patch(facecolor='lightblue', label='Data Transfer'),
        Patch(facecolor='lightgray', label='Waiting'),
        Patch(facecolor='blue', label='Processing')
    ]
    plt.legend(handles=legend_elements)
    plt.grid(True, axis='x', linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.savefig('schedule_gantt.png')
    plt.show()

def plot_resource_utilization(nodes):
    plt.figure(figsize=(14, 6))

    # Calculate resource utilization
    cpu_util = [n.busy_time / n.current_time * 100 if n.current_time > 0 else 0 for n in nodes]
    ram_util = [100 - (n.available_ram / n.total_ram * 100) for n in nodes]
    storage_util = [100 - (n.available_storage / n.total_storage * 100) for n in nodes]

    x = np.arange(len(nodes))
    width = 0.25

    plt.bar(x - width, cpu_util, width, label='CPU')
    plt.bar(x, ram_util, width, label='RAM')
    plt.bar(x + width, storage_util, width, label='Storage')

    plt.xlabel('Node ID')
    plt.ylabel('Utilization %')
    plt.title('Resource Utilization Across Nodes')
    plt.xticks(x, [n.id for n in nodes])
    plt.legend()
    plt.grid(True, axis='y', linestyle='--')
    plt.tight_layout()
    plt.savefig('resource_utilization.png')
    plt.show()

def plot_waiting_time_distribution(nodes):
    plt.figure(figsize=(10, 6))

    all_waiting_times = []
    for node in nodes:
        all_waiting_times.extend(node.waiting_times)

    if all_waiting_times:
        plt.hist(all_waiting_times, bins=20, color='skyblue', edgecolor='black')
        plt.axvline(np.mean(all_waiting_times), color='red', linestyle='dashed',
                   linewidth=2, label=f'Mean: {np.mean(all_waiting_times):.2f} sec')

        plt.xlabel('Waiting Time (seconds)')
        plt.ylabel('Task Count')
        plt.title('Task Waiting Time Distribution')
        plt.legend()
        plt.grid(True, axis='y', linestyle='--', alpha=0.7)
        plt.tight_layout()
        plt.savefig('waiting_time_dist.png')
        plt.show()

if __name__ == "__main__":
    simulate_cloud_fog()